In [29]:
import numpy as np
import cupy as cp
print(cp.__version__)
import tifffile as tiff
from skimage.morphology import skeletonize  # Import for skeletonization
from skan import Skeleton, summarize
from joblib import Parallel, delayed
import sys
from ltedt import local_thickness

13.3.0


In [4]:
binary_volume = tiff.imread("/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume.tif")
# binary_volume = tiff.imread("C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume.tif")
# Get the size of the volume
print(f'Size of the volume is: {binary_volume.shape}')

Size of the volume is: (1035, 1706, 1510)


In [5]:
# Cut the volume in a smaller part
binary_volume_bottom_part = binary_volume[300:500,600:800,200:400]

# output_path = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume_smaller.tif'
output_path = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller.tif'
tiff.imwrite(output_path, binary_volume_bottom_part)

In [6]:
smaller_skeleton = skeletonize(binary_volume_bottom_part)

# binarize the skeleton
smaller_skeleton = np.where(smaller_skeleton == True, 1, 0)

smaller_skeleton = smaller_skeleton.astype(np.uint8)


# output_path = 'C:/Users/nerea/Documents/MasterDTU/SpacialCourse_Fall24/specialCourse_fall24/outputs/output_volume_smaller_skeleton.tif'
output_path = '/zhome/57/0/203104/specialCourse_fall24/outputs/output_volume_smaller_skeleton.tif'
tiff.imwrite(output_path, smaller_skeleton)

In [7]:
# Implement thickness with cuda
thickness = local_thickness(binary_volume_bottom_part, implementation="cupy")


In [8]:
# Skeleton mask
thickness_map = thickness*smaller_skeleton


# Calcular el valor máximo en thickness_map, excluyendo los ceros
max_distance = np.max(thickness_map[thickness_map > 0])

print(f"Max distance in thickness map: {max_distance}")

Max distance in thickness map: 6


In [9]:
principal_skel = Skeleton(smaller_skeleton)
principal_branch_data = summarize(principal_skel)

branch_type_0 = principal_branch_data[principal_branch_data['branch-type'] == 0]
branch_type_1 = principal_branch_data[principal_branch_data['branch-type'] == 1]
branch_type_2 = principal_branch_data[principal_branch_data['branch-type'] == 2]
branch_type_3 = principal_branch_data[principal_branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


/tmp/ipykernel_2882629/3061349273.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  principal_branch_data = summarize(principal_skel)


Number of branch type 0: 0
Number of branch type 1: 67
Number of branch type 2: 64
Number of branch type 3: 0


## Branch 1 type delete 40 pixels length

In [10]:
import numpy as np

def neighbors(coord):
    x, y, z = coord
    return [
        # 6 primary (axis-aligned) neighbors
        (x.item() + 1, y.item(), z.item()), (x.item() - 1, y.item(), z.item()), 
        (x.item(), y.item() + 1, z.item()), (x.item(), y.item() - 1, z.item()),
        (x.item(), y.item(), z.item() + 1), (x.item(), y.item(), z.item() - 1),
        
        # 12 diagonal neighbors (changing two axes)
        (x.item() + 1, y.item() + 1, z.item()), (x.item() - 1, y.item() - 1, z.item()),
        (x.item() + 1, y.item(), z.item() + 1), (x.item() - 1, y.item(), z.item() - 1),
        (x.item(), y.item() + 1, z.item() + 1), (x.item(), y.item() - 1, z.item() - 1),
        (x.item() + 1, y.item() - 1, z.item()), (x.item() - 1, y.item() + 1, z.item()),
        (x.item() + 1, y.item(), z.item() - 1), (x.item() - 1, y.item(), z.item() + 1),
        (x.item(), y.item() + 1, z.item() - 1), (x.item(), y.item() - 1, z.item() + 1)
    ]

# Define the is_junction_point function to identify junction points
def is_junction_point(coord, skeleton):
    neighboring_coords = neighbors(coord)
    counter = 0
    for neigh in neighboring_coords:
        if skeleton[neigh[0], neigh[1], neigh[2]].item() > 0:
            counter = counter + 1
    
    if counter > 2:
        return True
    else:
        return False

    # skeleton_neighbors = sum([skeleton[tuple(neigh)] > 0 for neigh in neighboring_coords])
    # return skeleton_neighbors.item() > 2  # A junction point has more than 2 neighbors

# Define the main function to remove branches of type 1 based on the criteria
def remove_branches(skeleton, branch_data, skel, thickness_map, length_threshold, thickness_threshold):
    """
    Removes branches of type 1 from the skeleton based on length and thickness criteria,
    while preserving junction points.
    
    Parameters:
    skeleton (np.array): The 3D skeleton array to clean.
    branch_data (pd.DataFrame): DataFrame with branch information including branch type and distance.
    skel (Skeleton): Skeleton object to access branch coordinates.
    thickness_map (np.array): Array containing thickness values for each voxel.
    length_threshold (float): Maximum length for branches to consider for deletion.
    thickness_threshold (float): Minimum thickness for branches to consider for deletion.
    
    Returns:
    np.array: A cleaned version of the skeleton with specified branches removed.
    """
    # Copy the skeleton to avoid modifying the original
    skeleton_cleaned = np.copy(skeleton)
    
    # Iterate over each branch
    for branch_id in branch_data.index:
        # Only process branches of type 1
        if branch_data.loc[branch_id, 'branch-type'] == 1:
            branch_length = branch_data.loc[branch_id, 'branch-distance']
            
            # Check length condition
            if branch_length <= length_threshold:
                coordinates = skel.path_coordinates(branch_id)
                
                thickness_values = []
                for coord in coordinates:
                    thick_val = thickness_map[coord[0], coord[1], coord[2]]
                    thickness_values.append(thick_val.item())

                # Check thickness condition
                if min(thickness_values) <= thickness_threshold:
                    # Remove the branch, skipping junction points
                    for coord in coordinates:
                        # rounded_coord = tuple(np.round(coord).astype(int))
                        if not is_junction_point(coord, skeleton):
                            skeleton_cleaned[coord[0], coord[1], coord[2]] = 0
    
    return skeleton_cleaned


def print_branches(skeleton_cleaned):
    skel = Skeleton(skeleton_cleaned)
    branch_data = summarize(skel)

    branch_type_0 = branch_data[branch_data['branch-type'] == 0]
    branch_type_1 = branch_data[branch_data['branch-type'] == 1]
    branch_type_2 = branch_data[branch_data['branch-type'] == 2]
    branch_type_3 = branch_data[branch_data['branch-type'] == 3]

    print(f"Number of branch type 0: {len(branch_type_0)}")
    print(f"Number of branch type 1: {len(branch_type_1)}")
    print(f"Number of branch type 2: {len(branch_type_2)}")
    print(f"Number of branch type 3: {len(branch_type_3)}")


In [11]:
x = 40
thickness_threshold = 4

skeleleton_clean_x40_th4 = remove_branches(skeleton=smaller_skeleton, 
                                           branch_data=principal_branch_data, 
                                           skel=principal_skel, 
                                           thickness_map=thickness_map, 
                                           length_threshold=x, 
                                           thickness_threshold=thickness_threshold)
print_branches(skeleleton_clean_x40_th4)

output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/skeleleton_clean_x40_th4.tif'
tiff.imwrite(output_path_cleaned, skeleleton_clean_x40_th4)

Number of branch type 0: 6
Number of branch type 1: 31
Number of branch type 2: 29
Number of branch type 3: 2


/tmp/ipykernel_2882629/4125157522.py:84: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [12]:
x = 20
thickness_threshold = 5

skeleleton_clean_x20_th5 = remove_branches(skeleton=skeleleton_clean_x40_th4, 
                                           branch_data=principal_branch_data, 
                                           skel=principal_skel, 
                                           thickness_map=thickness_map, 
                                           length_threshold=x, 
                                           thickness_threshold=thickness_threshold)
print_branches(skeleleton_clean_x20_th5)

output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/skeleleton_clean_x20_th5.tif'
tiff.imwrite(output_path_cleaned, skeleleton_clean_x20_th5)

/tmp/ipykernel_2882629/4125157522.py:84: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


Number of branch type 0: 11
Number of branch type 1: 27
Number of branch type 2: 17
Number of branch type 3: 1


In [13]:
skel = Skeleton(skeleleton_clean_x20_th5)
branch_data = summarize(skel)

for branch_id in branch_data.index:
    if branch_data.loc[branch_id, 'branch-type'] == 0:
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        print(branch_length)

48.35873884767937
12.949382989376325
1.4142135623730951
3.82842712474619
1.7320508075688772
3.82842712474619
7.706742302257039
21.070338854006465
2.8284271247461903
11.217332181807448
2.8284271247461903


/tmp/ipykernel_2882629/2796071380.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [25]:
import cc3d
import numpy as np

labels_out = cc3d.connected_components(skeleleton_clean_x20_th5)
labels_out, N = cc3d.connected_components(skeleleton_clean_x20_th5, return_N=True)
print(N)

22


In [28]:
for branch_id in branch_data.index:
    branch_blob_mapping = {}

    coordinates = skel.path_coordinates(branch_id)
    branch_labels = labels_out[coordinates[:, 0], coordinates[:, 1], coordinates[:, 2]]

    # Find the unique label(s) associated with this branch
    unique_blob_labels = np.unique(branch_labels)
    
    # If there is more than one label, we need to assign multiple blobs, otherwise one blob
    if len(unique_blob_labels) > 1:
        branch_blob_mapping[branch_id] = unique_blob_labels
    else:
        branch_blob_mapping[branch_id] = unique_blob_labels[0]

In [26]:
import numpy as np

# Assuming `skeleleton_clean_x20_th5` is your skeleton (a 3D binary image) and
# `labels_out` is the output from connected components, which gives the blob labels

def assign_blob_to_branches(skeleton, labels_out):
    # Initialize a dictionary to store the blob for each branch
    branch_blob_mapping = {}
    
    # Iterate over all the branches (assuming you have a way to identify branches in the skeleton)
    for branch_id in range(len(principal_branch_data)):  # or use a different identifier for branches
        # Get the coordinates of the branch voxels (you need to identify these from the branch data)
        branch_voxels = principal_branch_data[branch_id]  # assuming it's a list of voxel coordinates
        
        # Get the unique connected component label for the branch's voxels
        # Extract the labels for the voxels belonging to this branch
        branch_labels = labels_out[branch_voxels[:, 0], branch_voxels[:, 1], branch_voxels[:, 2]]
        
        # Find the unique label(s) associated with this branch
        unique_blob_labels = np.unique(branch_labels)
        
        # If there is more than one label, we need to assign multiple blobs, otherwise one blob
        if len(unique_blob_labels) > 1:
            branch_blob_mapping[branch_id] = unique_blob_labels
        else:
            branch_blob_mapping[branch_id] = unique_blob_labels[0]
    
    return branch_blob_mapping

# Get the branch to blob mapping
branch_blob_mapping = assign_blob_to_branches(skeleleton_clean_x20_th5, labels_out)

# Print the result
for branch_id, blob_id in branch_blob_mapping.items():
    print(f"Branch {branch_id} belongs to blob(s) {blob_id}")


KeyError: 0

## Codigo pasado!

In [15]:
x = 40  # Número mínimo de píxeles de la rama
thickness_threshold = 4  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_2 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_2[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_2 = skeleton_cleaned_2.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_2.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_2)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [16]:
skel = Skeleton(skeleton_cleaned_2)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


Number of branch type 0: 6
Number of branch type 1: 59
Number of branch type 2: 50
Number of branch type 3: 0


/tmp/ipykernel_2882629/1015276533.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [17]:
x = 60  # Número mínimo de píxeles de la rama
thickness_threshold = 6  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_3 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_3[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_3 = skeleton_cleaned_3.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_3.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_3)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [18]:
skel = Skeleton(skeleton_cleaned_3)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")


Number of branch type 0: 16
Number of branch type 1: 29
Number of branch type 2: 17
Number of branch type 3: 0


/tmp/ipykernel_2882629/1867985952.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [19]:
x = 40  # Número mínimo de píxeles de la rama
thickness_threshold = 6  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_4 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_4[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_4 = skeleton_cleaned_4.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_4.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_4)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [20]:
skel = Skeleton(skeleton_cleaned_4)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")

Number of branch type 0: 11
Number of branch type 1: 52
Number of branch type 2: 34
Number of branch type 3: 0


/tmp/ipykernel_2882629/3158189669.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)


In [21]:
x = 40  # Número mínimo de píxeles de la rama
thickness_threshold = 5  # Umbral de distancia en el thickness_map


# Initialize mask for cleaning only type 1 branches that meet the criteria
skeleton_cleaned_5 = np.copy(smaller_skeleton)

# Loop through each branch of type 1 to assess whether it should be deleted
for branch_id in branch_data.index:
    # Check if this branch is of type 1
    if branch_data.loc[branch_id, 'branch-type'] == 1:
        # Get branch length
        branch_length = branch_data.loc[branch_id, 'branch-distance']
        
        # Only consider branches that are shorter than the length threshold
        if branch_length < x:
            # Get the coordinates of each voxel in this branch
            coordinates = skel.path_coordinates(branch_id)
            
            # Check the thickness values at these coordinates in thickness_map
            thickness_values = [thickness_map[tuple(np.round(coord).astype(int))] for coord in coordinates]
            
            # If the minimum thickness in this branch is below the threshold, mark for deletion
            if min(thickness_values) < thickness_threshold:
                # Set the corresponding pixels in the skeleton to 0
                for coord in coordinates:
                    skeleton_cleaned_5[tuple(np.round(coord).astype(int))] = 0  # Ensure coordinates are integers

# Ensure skeleton_cleaned is in a compatible data type for TIFF files
skeleton_cleaned_5 = skeleton_cleaned_5.astype(np.uint8)

# Save the cleaned skeleton to a file
output_path_cleaned = '/zhome/57/0/203104/specialCourse_fall24/outputs/smaller_skeleton_cleaned_5.tif'
tiff.imwrite(output_path_cleaned, skeleton_cleaned_5)

print("El esqueleto ha sido limpiado y guardado en el archivo.")


El esqueleto ha sido limpiado y guardado en el archivo.


In [22]:
skel = Skeleton(skeleton_cleaned_5)
branch_data = summarize(skel)

branch_type_0 = branch_data[branch_data['branch-type'] == 0]
branch_type_1 = branch_data[branch_data['branch-type'] == 1]
branch_type_2 = branch_data[branch_data['branch-type'] == 2]
branch_type_3 = branch_data[branch_data['branch-type'] == 3]

print(f"Number of branch type 0: {len(branch_type_0)}")
print(f"Number of branch type 1: {len(branch_type_1)}")
print(f"Number of branch type 2: {len(branch_type_2)}")
print(f"Number of branch type 3: {len(branch_type_3)}")

Number of branch type 0: 17
Number of branch type 1: 36
Number of branch type 2: 21
Number of branch type 3: 0


/tmp/ipykernel_2882629/2138058238.py:2: VisibleDeprecationWarning: separator in column name will change to _ in version 0.13; to silence this warning, use `separator='-'` to maintain current behavior and use `separator='_'` to switch to the new default behavior.
  branch_data = summarize(skel)
